<a target="_blank" href="https://colab.research.google.com/github/Sebelino/DD2424-project/blob/main/e1_sebastian.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [10]:
%load_ext autoreload
%autoreload 2

import os
import socket

repo_name = "DD2424-project"
dep_filename = "download_dataset.py"
cwd = os.getcwd()
hostname = socket.gethostname()

print(f"Host: {hostname}")
print(f"You are here: {cwd}")

if os.path.exists(dep_filename):
    print(f"{dep_filename} is in your current directory. Assuming you are in the root of the repository.")
elif not os.path.exists(repo_name):
    print(f"Cloning repo and fetching {dep_filename}...")
    !git clone https://github.com/Sebelino/DD2424-project
    !cp DD2424-project/download_dataset.py .
else:
    !git -C DD2424-project pull --rebase
    !cp DD2424-project/download_dataset.py .

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Host: deeplearning-vm
You are here: /home/sebelino/pycharm_deployment
download_dataset.py is in your current directory. Assuming you are in the root of the repository.


In [11]:
from determinism import Determinism

determinism = Determinism(seed=42).sow()  # Keep this before any torch import
print("Training results should now be reproducible.")


Training results should now be reproducible.


# Train model

In [ ]:
import json
from training import TrainParams, NagParams
from datasets import DatasetParams
from search import run_search

dataset_params = DatasetParams(
    splitting_seed=determinism.seed,
    shuffler_seed=determinism.seed,
    batch_size=32,
    trainval_size=None,
    #trainval_size=400,
    validation_set_fraction=0.2,
)

training_params = TrainParams(
    seed=determinism.seed,
    architecture="resnet50",
    n_epochs=10,
    optimizer=NagParams(
        learning_rate=None,  # Included in search
        weight_decay=None,  # Included in search
        momentum=None,  # Included in search
    ),
    freeze_layers=True,
    unfreezing_epochs=(None, None),  # Included in search
    validation_freq=1,
    time_limit_seconds=None,
    val_acc_target=None,
)

search_result = run_search(dataset_params, training_params, n_trials=3)

print(json.dumps(search_result, indent=2))



[I 2025-05-07 10:43:51,439] A new study created in memory with name: no-name-1a8cbb75-cdc4-461e-9c4f-a6a6fcb8de9b


Running trial with training parameters:
{
  "seed": 42,
  "architecture": "resnet50",
  "optimizer": {
    "learning_rate": 0.00031489116479568613,
    "weight_decay": 0.0007114476009343421,
    "momentum": 0.9122782431253075,
    "name": "nag"
  },
  "n_epochs": 10,
  "freeze_layers": true,
  "unfreezing_epochs": [3, 4],
  "validation_freq": 1
}


Update step:   0%|          | 0/92 [00:00<?, ?it/s]